In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("./lending_data.csv")
lending_df = pd.read_csv(file_path)
# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]

# Separate the X variable, the features
X = lending_df.drop(columns="loan_status")


In [4]:
# Review the y variable Series
y[0:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 7)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

# Fit the model using training data
lr_model=logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test)
results = pd.DataFrame({"Prediction": testing_predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
from sklearn.metrics import accuracy_score
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, testing_predictions))

0.9442676901753825


In [11]:
# Import the model for sklearn confusion matrix
from sklearn.metrics import confusion_matrix

# Create and save the confusion matrix for the training data
training_predictions=lr_model.predict(X_train)
training_matrix = confusion_matrix(y_train, training_predictions)
# Print the confusion matrix for the training data
print(training_matrix)

[[55980   297]
 [  198  1677]]


In [13]:
# Create and save the training classification report
target_names = ["Healthy Loan", "High Risk Loan"]
training_report = classification_report(y_train, training_predictions, target_names=target_names)

# Print the training classification report
print(training_report)

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     56277
High Risk Loan       0.85      0.89      0.87      1875

      accuracy                           0.99     58152
     macro avg       0.92      0.94      0.93     58152
  weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model was 99% accurate in predicting healthy loans and unhealthy loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_ros_model, y_ros_model = ros.fit_resample(X_train,y_train)

In [15]:
from collections import Counter
# Count the distinct values of the resampled labels data
print(Counter(X_ros_model))
print(Counter(y_ros_model))

Counter({'loan_size': 1, 'interest_rate': 1, 'borrower_income': 1, 'debt_to_income': 1, 'num_of_accounts': 1, 'derogatory_marks': 1, 'total_debt': 1})
Counter({0: 56277, 1: 56277})


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_ros_model, y_ros_model)
# Make a prediction using the testing data
predictions = classifier.predict(X_ros_model)
pd.DataFrame({'Predictions': predictions, 'Actual': y_ros_model}).head()

,Predictions,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model 
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_ros_model, predictions)}")

The balanced accuracy score of the model is: 0.994180571103648


In [18]:
# Generate a confusion matrix for the model
cf_matrix = confusion_matrix(y_ros_model, predictions)
cf_matrix 

array([[55945,   332],
       [  323, 55954]])

In [19]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High Risk Loan"]
report = classification_report(y_ros_model, predictions, target_names=target_names)
print(report)

                precision    recall  f1-score   support

  Healthy Loan       0.99      0.99      0.99     56277
High Risk Loan       0.99      0.99      0.99     56277

      accuracy                           0.99    112554
     macro avg       0.99      0.99      0.99    112554
  weighted avg       0.99      0.99      0.99    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the oversampled data with near-perfect accuracy (>99% accurate)